https://qpleple.com/topic-coherence-to-evaluate-topic-models/

https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf

https://www.kaggle.com/code/canggih/topic-modeling/notebook

Topic modeling is a statistical technique used to uncover hidden "topics" within a collection of documents. While it is traditionally applied to text documents, it has recently gained prominence in social media analysis. One of the most widely used algorithms for this task is Latent Dirichlet Allocation (LDA), introduced by David Blei et al. in 2003. In this project, I aim to apply topic modeling to the upvoted Kaggle dataset to identify underlying themes.

In [3]:
!pip install stop-words


  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=c1d83fb99c0b57efafa3998dd22ce1124312c9a32efff48c61a12304086c8411
  Stored in directory: /root/.cache/pip/wheels/8f/a5/51/a5405e1da5d178491b79d12cc81b6cb9bb14fe2c8c632eba70
Successfully built stop-words


In [8]:
!pip install pyLDAvis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.9 MB/s eta 0:00:00


In [9]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim
import pprint
import nltk

In [23]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()


In [12]:
df = pd.read_csv('/content/voted-kaggle-dataset.csv')

In [13]:
df.head()

,Title,Subtitle,Owner,Votes,Last Update,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB,1233,2016-11-05,crime\nfinance,CSV,144 MB,ODbL,"440,221 views","52,793 downloads","1,778 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,The datasets contains transactions made by cre...
1,European Soccer Database,"25k+ matches, players & teams attributes for E...",Hugo Mathien,1035,2016-10-24,association football\neurope,SQLite,299 MB,ODbL,"393,924 views","46,025 downloads","1,459 kernels",75 topics,https://www.kaggle.com/hugomathien/soccer,The ultimate Soccer database for data analysis...
2,TMDB 5000 Movie Dataset,"Metadata on ~5,000 movies from TMDb",The Movie Database (TMDb),1018,2017-09-28,film,CSV,44 MB,Other,"444,535 views","61,705 downloads","1,394 kernels",46 topics,https://www.kaggle.com/tmdb/tmdb-movie-metadata,Background\nWhat can we say about the success ...
3,Human Resources Analytics,Why are our best and most experienced employee...,Ludovic Benistant,832,2016-11-30,employment,CSV,553 KB,CC4,"309,644 views","47,350 downloads","1,772 kernels",32 topics,https://www.kaggle.com/ludobenistant/hr-analytics,This dataset is simulated\nWhy are our best an...
4,Global Terrorism Database,"More than 170,000 terrorist attacks worldwide,...",START Consortium,785,2017-07-19,crime\nterrorism\ninternational relations,CSV,144 MB,Other,"186,621 views","26,091 downloads",609 kernels,11 topics,https://www.kaggle.com/START-UMD/gtd,"Context\nInformation on more than 170,000 Terr..."


In [14]:
print(df['Description'].head(1))

0    The datasets contains transactions made by cre...
Name: Description, dtype: object


In [24]:
removed_words = {'http', 'https', 'amp', 'rt', 't', 'c'}

In [25]:

nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
text = []
for i in df['Description'].items():
  # clean and tokenize document string
  raw = str(i[1]).lower()
  tokens = tokenizer.tokenize(raw)

  stopped_tokens = [raw for raw in tokens if not raw in en_stop] # remove stop words from tokens

  stopped_tokens_new = [raw for raw in stopped_tokens if raw not in removed_words] # remove stop words from tokens

  lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]  # lemmatize tokens

  new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw)==1]# remove word containing only single char

  text.append(new_lemma_tokens)# add tokens to list

print(text[0])






['datasets', 'contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'dataset', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'dataset', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'provide', 'original', 'feature', 'background', 'information', 'data', 'feature', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'can', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring', 'accuracy', 'using'

In [27]:
print("Before Removing Extra Words:", stopped_tokens)
print("After Removing Extra Words:", stopped_tokens_new)


Before Removing Extra Words: ['dataset', 'description', 'yet']
After Removing Extra Words: ['dataset', 'description', 'yet']


In [30]:
dictiionary = corpora.Dictionary(text) # turn our tokenized documents into a id <-> term dictionary
corpus = [dictiionary.doc2bow(text) for text in text] # convert tokenized documents into a document-term matrix

print(corpus[:5])

[[(0, 3), (1, 1), (2, 2), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 3), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 3), (38, 4), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 7), (50, 1), (51, 4), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 3), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 7), (105, 1), (106, 1), (107, 1), (108, 4), (109, 3), (110, 1)

Generate LDA model

([
  (probability, 'word1'),

  (probability, 'word2'),

  (probability, 'word3'),

  (probability, 'word4'),

  (probability, 'word5')
  
], coherence_score)


The probability next to each word indicates how strongly the word is associated with that topic.

The coherence score (negative value) tells us how well the words in the topic make sense together.

Higher coherence scores (closer to 0) mean that the topic is well-defined, and the words make sense together.

Lower coherence scores (more negative) suggest that the topic might be too noisy or incoherent, and we might need to improve the model.

In [31]:
LDAmodel = gensim.models.ldamodel.LdaModel(corpus, num_topics= 15, id2word= dictiionary, passes= 20)



pprint.pprint(LDAmodel.top_topics(corpus, topn = 5))

[([(0.051117804, 'data'),
   (0.020044971, 'others'),
   (0.017847534, 'content'),
   (0.017666193, 'acknowledgement'),
   (0.017629938, 'context')],
  -0.4481374056571769),
 ([(0.029037476, 'data'),
   (0.010982911, 'dataset'),
   (0.009805847, 'year'),
   (0.009257687, 'can'),
   (0.0057570348, 'content')],
  -0.6780837681589124),
 ([(0.024354832, 'word'),
   (0.02134403, 'language'),
   (0.011664786, 'corpus'),
   (0.01138582, 'text'),
   (0.008676317, 'english')],
  -1.0465451029130652),
 ([(0.09703695, 'university'),
   (0.016541809, 'license'),
   (0.015104636, 'package'),
   (0.015103535, 'context'),
   (0.014472429, 'id')],
  -1.152360454647618),
 ([(0.029573057, 'model'),
   (0.023674916, 'trained'),
   (0.017663782, 'dataset'),
   (0.014048313, 'pre'),
   (0.01360316, 'image')],
  -1.3157534287355774),
 ([(0.017198967, 'state'),
   (0.014580425, 'data'),
   (0.007905465, 'dataset'),
   (0.006408332, 'open'),
   (0.0060515325, 'code')],
  -1.37526848264644),
 ([(0.020862473, '

In [32]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(LDAmodel,corpus, dictiionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.206780  0.039708       1        1  24.556024
10    -0.192205  0.060866       2        1  23.910695
3     -0.160501  0.031007       3        1  12.277098
0     -0.057026 -0.033373       4        1   6.875623
8     -0.059768 -0.029403       5        1   5.123764
9     -0.087544  0.005969       6        1   4.427941
11    -0.063893  0.034050       7        1   3.881974
12    -0.033629  0.045791       8        1   3.640921
7      0.032269 -0.074120       9        1   2.762863
5      0.007871  0.005328      10        1   2.711379
6      0.129113 -0.069062      11        1   2.354967
2      0.264328  0.325078      12        1   2.166523
14     0.139509 -0.111775      13        1   1.858771
13     0.117161 -0.106789      14        1   1.746530
1      0.171096 -0.123275      15        1   1.704927, topic_info=         Term         Freq        Total Category  logprob  loglift
38    dataset  4451.000000  4451.000000  Default  30.0000  30.0000
956      name  1630.000000  1630.000000  Default  29.0000  29.0000
146       com  1423.000000  1423.000000  Default  28.0000  28.0000
289      type  1179.000000  1179.000000  Default  27.0000  27.0000
37       data  6939.000000  6939.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1327   camera    17.568763    45.222286  Topic15  -5.8079   3.1262
1217     text    45.997735   612.330226  Topic15  -4.8455   1.4830
538    number    31.488501  1442.316042  Topic15  -5.2244   0.2473
115     value    20.944165   646.462055  Topic15  -5.6322   0.6420
2812  cluster    14.840392   103.904758  Topic15  -5.9767   2.1255

[1014 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
20575     14  0.809201             _f
20587     14  0.890459             _m
4557       2  0.960772  accelerometer
523        6  0.303857       accident
523        9  0.683678       accident
...      ...       ...            ...
3211       7  0.037814           zone
3211       9  0.037814           zone
3211      13  0.529401           zone
13411      6  0.875628             æg
16031     13  0.955859         índexs

[2785 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 11, 4, 1, 9, 10, 12, 13, 8, 6, 7, 3, 15, 14, 2])

In pyLDAvis, you see:

Bubbles (Topics): Size indicates prevalence, distance shows topic similarity.

Top Words for a Topic: Shows words that define a selected topic.

λ (Relevance Slider): Controls how words are ranked for interpretability.


It contains two key pieces of information:

The most probable words for the topic (along with their probabilities).

The coherence score of the topic, which measures how meaningful and interpretable the topic is.